In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Datasets/Augen
!ls



Mounted at /content/gdrive
/content/gdrive/MyDrive/Datasets/Augen
0  1  2


In [1]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split

# number of learnings should be until the loss converges otherwise we overfit
EPOCHS = 5
TOTAL_EPOCHS = EPOCHS + 7
IMG_WIDTH = 92
IMG_HEIGHT = 92
NUM_CATEGORIES = 3
# number of images getting used for testing not for training in percent
TEST_SIZE = 0.2
# drop out nodes at random to make the network more robust
DROPOUT = 0.4
NORMALIZE = True
HIDDENLAYERS_SIZE = 64
STEPS_PER_EPOCH = 1
BATCH_SIZE = 10
IMG_SIZE = (IMG_WIDTH, IMG_HEIGHT)
IMG_SHAPE = (IMG_HEIGHT,IMG_WIDTH,3)
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                                  include_top=False,
                                               weights='imagenet')
LEARNING_RATE = 0.001
# batchsize = x -> could define one // woundn't use it except if it takes to long
# it is the amount of images in the data set which 
# is getting looked at before an adjustment


def main():
    base_model.summary()
    # Get image arrays and labels for all image files
    images, labels = load_data(os.getcwd())
    print(labels)
    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )
    model = get_model()

    # grid search
    # model = KerasClassifier(build_fn = lambda:get_model())
    """
    batch_size = [1, 5, 10, 50, 100]
    epochs = [5, 10 , 20,  50, 100]
    dropout_rate = [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
    pool_type =  ['max', 'average']
    param_grid = dict(batch_size = batch_size, epochs = epochs)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3,
                        verbose = 2)
    grid_result = grid.fit(x_train, y_train)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_,
                                 grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

    """
    # Fit model on training data
    History = model.fit(x_train, y_train, epochs=EPOCHS)

     # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=1)


    # prepare the base model
    base_model.trainable = True

    # Let's take a look to see how many layers are in the base model
    print("Number of layers in the base model: ", len(base_model.layers))

    # Fine-tune from this layer onwards
    fine_tune_at = 100

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
      layer.trainable =  False
    model.summary()

    for layer in base_model.layers[fine_tune_at:]:
      if layer.name[-2:] == "BN" or layer.name[-2:] == "bn":
        layer.trainable = False
    model.summary()
    
    decayrate = 1
    
    #finds the last learning rate of the pretrained model
    base_learning_rate = LEARNING_RATE / (1 + decayrate * STEPS_PER_EPOCH /
                                          STEPS_PER_EPOCH*1000)

    #compile the model for finetuning
    model.compile(loss="categorical_crossentropy",
              optimizer = tf.keras.optimizers.RMSprop(
                  learning_rate=base_learning_rate),
              metrics="precision")

    #train the finetuned model with a smaler learningrate
    history_fine =  model.fit(x_train, y_train, 
                         epochs=TOTAL_EPOCHS,
                         initial_epoch=History.epoch[-1])

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=1)
    %cd ..
    %cd ..
    !ls 
    !mkdir /content/gdrive/MyDrive/Deep_CNN_Augen
    filename = ("/content/gdrive/MyDrive/Deep_CNN_Augen")
    !ls
    model.save(filename)
    %cd Datasets/Lachen

def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = list()
    labels = list()

    path = data_dir
    # loop over all the directories
    for dirpath, dirnames, filenames in os.walk(path):
        print("q")
        if(filenames != ['.DS_Store']):
            # loops over all the files inside the directory
            for curFile in filenames:
                if(curFile != '.DS_Store' and curFile != 'a'):
                    file = dirpath + os.sep + curFile
                    # reads the image
                    image = cv2.imread(file)
                    # rezeises the image
                    resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT),
                                        interpolation=cv2.INTER_AREA)
                    # make the input better for the neural network
                    if(NORMALIZE):
                        resized = resized / 255.0
                    # add to pictures
                    images.append(resized)
                    # add to labels
                    print(dirpath)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/closed-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/closed-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/closed-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/closed-input-0"):
                        labels.append(0)
                        print(dirpath)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/no_face-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/no_face-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/no_face-input-3" or 
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/no_face-input-0"):
                        labels.append(1)
                        print(dirpath)
                    if (dirpath == "/content/gdrive/My Drive/Datasets/Augen/3/open-input-3" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/2/open-input-2" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/1/open-input-1" or
                        dirpath == "/content/gdrive/My Drive/Datasets/Augen/0/open-input-0"):
                        labels.append(2)
                        print(dirpath)
    return(images, labels)

def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    # make the model more robust
    data_augmentation = tf.keras.Sequential(
      [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                    input_shape=(IMG_HEIGHT, 
                                                                  IMG_WIDTH,
                                                                  3)),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
      ]
    )

    # decrease the rate of change towards the end
    lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
        LEARNING_RATE,
        decay_steps=STEPS_PER_EPOCH*1000,
        decay_rate=1,
        staircase=False)
    
    # Create the base model from the pre-trained model MobileNet V2
    IMG_SHAPE = IMG_SIZE + (3,)
    base_model.trainable = False
    # bias
    #model with several layers
    model = tf.keras.models.Sequential([
        data_augmentation,                                
        # add the convolution layer -> extract feature such as strong colour difference
        #-> lernt Kanten
        base_model,
        # extract information
        tf.keras.layers.GlobalAveragePooling2D(),
        # add a dropout
        tf.keras.layers.Dropout(DROPOUT),
        # add the output layer -> softmax gives a probability distribution min loss = 0 max = infinity
        # continues until the end
        tf.keras.layers.Dense(NUM_CATEGORIES, "softmax")
    ])
    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr_schedule),
        loss="categorical_crossentropy",
        metrics=["precision"]
    )
    return model

if __name__ == "__main__":
  main()

9420800/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 92, 92, 3)]  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 46, 46, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 46, 46, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 46, 46, 32)   0           bn_Conv1[0][0]                   
________________

error: ignored